In [8]:
import geopandas as gpd
import importlib
import numpy as np
import os
import pandas as pd
import shapely.wkb
from shapely.geometry import Point
from sqlalchemy import create_engine, text

In [48]:
import utils
importlib.reload(utils)
from utils import verif

In [5]:
from dotenv import load_dotenv

load_dotenv()

DBNAME = os.getenv('DBNAME')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')
CONNEXION = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"

DBNAME_DVF_ANCIEN = os.getenv('DBNAME_DVF_ANCIEN')
DBNAME_ANCIEN = os.getenv('DBNAME_ANCIEN')
USER_ANCIEN = os.getenv('USER_ANCIEN')
PASSWORD_ANCIEN = os.getenv('PASSWORD_ANCIEN')
HOST_ANCIEN = os.getenv('HOST_ANCIEN')
PORT_ANCIEN = os.getenv('PORT_ANCIEN')
CONNEXION_ANCIEN = f"postgresql+psycopg2://{USER_ANCIEN}:{PASSWORD_ANCIEN}@{HOST_ANCIEN}:{PORT_ANCIEN}/{DBNAME_DVF_ANCIEN}"

PATH_DATA_G = os.getenv('PATH_DATA_G')
G_FILOSOFI = 'filosofi'
G_LOGEMENT = 'rp_logement'
G_DV3F = 'dv3f'

PATH_DATA_C = os.getenv('PATH_DATA_C')
C_CORRES = 'corres'

PATH_DATA_DV3F = 'dv3f'

python-dotenv could not parse statement starting at line 16


### CORRESPONDANCE

In [83]:
engine = create_engine(CONNEXION)
requete_corres_iris_com = text("""
                               SELECT DISTINCT 
                               inseeiris::int as code_iris, 
                               inseecom::int as code_com 
                               from hab.iris_100
                    """)
requete_corres_com_epci = text("""
                               SELECT DISTINCT 
                               insee_comm::int as code_com, 
                               siren_epci::int as code_epci 
                               from hab.v_epcifp_comm
                    """)

with engine.connect() as connection:
    result = connection.execute(requete_corres_iris_com)
    corres_iris_com = pd.DataFrame(result.fetchall(), columns=result.keys())

    result = connection.execute(requete_corres_com_epci)
    corres_com_epci = pd.DataFrame(result.fetchall(), columns=result.keys())

corres_iris_com_epci = pd.merge(corres_iris_com, corres_com_epci, on='code_com', how='left')
corres_iris_com_epci

,code_iris,code_com,code_epci
0,341720803,34172,243400017
1,341570101,34157,200066355
2,341540106,34154,243400470
3,341450107,34145,243400520
4,341440000,34144,200042646
...,...,...,...
573,340250000,34025,243400769
574,341580000,34158,200066348
575,341160102,34116,243400017
576,341560000,34156,243400355


### CODE / ECHELLE

In [41]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
select
    34 as code,
    'departement' as echelle
union
select 
    numero_siren::int as code,
    'epci' as echelle
from hab.v_epcifp_500 
union 
select 
    insee_com::int as code,
    'commune' as echelle
from hab.commune_500 
where insee_com like '34%' 
union
select 
    inseeiris::int as code,
    'iris' as echelle 
from hab.iris_100
order by code
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    code = pd.DataFrame(result.fetchall(), columns=result.keys())

code['echelle'].value_counts()

echelle
iris           578
commune        341
epci            17
departement      1
Name: count, dtype: int64

### ZONAGE GEOGRAPHQUE

In [36]:
# Création de l'engine SQLAlchemy
engine = create_engine(CONNEXION)

requete = text(
    """
    select *
    from hab.v_epcifp_500 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_epci = pd.DataFrame(result.fetchall(), columns=result.keys())


requete = text(
    """
    select *
    from hab.commune_500 
    where insee_com like '34%' 
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    zonage_com = pd.DataFrame(result.fetchall(), columns=result.keys())


requete = text(
    """
    select *
    from hab.iris_100
    """
    )

# Tester la connexion
with engine.connect() as connection:
    result = connection.execute(requete)
    rows = result.fetchall()
    columns = result.keys()
    dict_rows = [dict(zip(columns, row)) for row in rows]
    for row in dict_rows:
        row['shape'] = shapely.wkb.loads(row['shape'], hex=True)
    zonage_iris = gpd.GeoDataFrame(dict_rows, geometry='shape', crs="EPSG:2154")
    zonage_iris = zonage_iris.to_crs(epsg=4326) # WGS 84 -> Lambert 93
    zonage_iris['inseeiris'] = zonage_iris['inseeiris'].astype(int)


## DECRET

In [49]:
G_DECRET = 'decret'
chemin = os.path.join(PATH_DATA_G, G_DECRET, "2023_decret.xlsx")
decret = pd.read_excel(chemin, sheet_name="3")
verif(decret, 'INSEE')
decret.head(2)

Hérault non présent
EPCI non présentes
83 communes présentes
Le dataframe ne contient pas de colonnes vides
IRIS non présentes


,agglo_decret,INSEE,décret
0,Sète,34023,1
1,Sète,34024,1


## AIRDNA

In [33]:
G_AIRDNA = 'airdna'

chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2022.xlsx')
bnb2022 = pd.read_excel(chemin)
bnb2022 = bnb2022[bnb2022['Liste logement']=='Entire home/apt']
bnb2022 = bnb2022[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel €',
                             'Nb jours réservés','Nb total jours de disponibilité']]
bnb2022.columns = ['code_com','com','code_epci','epci','longitude','latitude','rev_annuel',
                   'nb_jours_reserv','nb_jours_dispo']

chemin = os.path.join(PATH_DATA_G, G_AIRDNA, 'offre_airdna_34_2023.xlsx')
bnb2023 = pd.read_excel(chemin)
bnb2023 = bnb2023[bnb2023['Nature logement']=='Entire home/apt']
bnb2023 = bnb2023[['INSEE','Communes','Code EPCI','EPCI','Longitude','Latitude','Revenu annuel â‚¬',
                             'Nb jours rÃ©servÃ©s','Nb total jours de disponibilitÃ©']]
bnb2023.columns = ['code_com','com','code_epci','epci','longitude','latitude','rev_annuel',
                   'nb_jours_reserv','nb_jours_dispo']

In [38]:
def traitement_bnb(bnb, code):
    """
    Fonction de traitement des données Airbnb
    """
    ### jointure IRIS
    bnb['geometry'] = bnb.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
    bnb_gdf = gpd.GeoDataFrame(bnb, geometry='geometry', crs="EPSG:4326")
    bnb_gdf = gpd.sjoin(bnb_gdf, zonage_iris, how="left", predicate="within")

    ### vérification de la jointure
    bnb_gdf.loc[:, "inseeiris_str"] = bnb_gdf["inseeiris"].astype(str)
    bnb_gdf.loc[:, "inseeiris_com"] = bnb_gdf["inseeiris_str"].str[:5]
    bnb_gdf.loc[:, "code_com_str"] = bnb_gdf["code_com"].astype(str)
    bnb_ss_iris_diff_iris = bnb_gdf[bnb_gdf["inseeiris_com"] == bnb_gdf["code_com_str"]]
    print(f"Nombre de logements Airbnb sans IRIS : {len(bnb_gdf) - len(bnb_ss_iris_diff_iris)}")

    ### agrégation
    bnb_ss_iris_diff_iris.loc[:, 'code_dep'] = 34
    bnb_ss_iris_diff_iris_agg = bnb_ss_iris_diff_iris.groupby([code]).agg(
        nombre_reserv_120=('nb_jours_reserv', lambda x: (x >= 120).sum()),
        nombre_dispo_120=('nb_jours_dispo', lambda x: (x >= 120).sum()),
        nombre_total=('nb_jours_reserv', 'count'),
        jours_reserves=('nb_jours_reserv', 'sum'),
        revenu=('rev_annuel', 'sum'),
        jours_reserves_120=('nb_jours_reserv', lambda x: x[bnb_ss_iris_diff_iris.loc[x.index, 'nb_jours_dispo'] >= 120].sum()),
        revenu_120=('rev_annuel', lambda x: x[bnb_ss_iris_diff_iris.loc[x.index, 'nb_jours_dispo'] >= 120].sum())
    ).reset_index()
    bnb_ss_iris_diff_iris_agg.loc[:, 'code'] = bnb_ss_iris_diff_iris_agg[code].astype(int)
    bnb_ss_iris_diff_iris_agg.drop(columns=[code], inplace=True)

    return bnb_ss_iris_diff_iris_agg


In [39]:
bnb_iris_2022 = traitement_bnb(bnb2022, 'inseeiris')
bnb_iris_2023 = traitement_bnb(bnb2023, 'inseeiris')

bnb_com_2022 = traitement_bnb(bnb2022, 'code_com')
bnb_com_2023 = traitement_bnb(bnb2023, 'code_com')

bnb_epci_2022 = traitement_bnb(bnb2022, 'code_epci')
bnb_epci_2023 = traitement_bnb(bnb2023, 'code_epci')

bnb_dep_2022 = traitement_bnb(bnb2022, 'code_dep')
bnb_dep_2023 = traitement_bnb(bnb2023, 'code_dep')

Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165
Nombre de logements Airbnb sans IRIS : 194
Nombre de logements Airbnb sans IRIS : 165


In [42]:
bnb_2022 = pd.concat([bnb_dep_2022, bnb_epci_2022, bnb_com_2022, bnb_iris_2022], axis=0).drop_duplicates().reset_index(drop=True)
bnb_2023 = pd.concat([bnb_dep_2023, bnb_epci_2023, bnb_com_2023, bnb_iris_2023], axis=0).drop_duplicates().reset_index(drop=True)

bnb_final = pd.merge(bnb_2022, bnb_2023, how='outer', on='code', suffixes=('_2022', '_2023'))

bnb_final_final = pd.merge(code, bnb_final, how='left', on='code')

In [43]:
# Calculer les évolutions et les écarts
bnb_final_final_final = bnb_final_final.assign(
    evol_bnb_120=(bnb_final_final['nombre_dispo_120_2023'] - bnb_final_final['nombre_dispo_120_2022']) / bnb_final_final['nombre_dispo_120_2022'],
    ecart_bnb_120=bnb_final_final['nombre_dispo_120_2023'] - bnb_final_final['nombre_dispo_120_2022'],
    ecart_bnb_tot=bnb_final_final['nombre_total_2023'] - bnb_final_final['nombre_total_2022'],
    evol_bnb_tot=(bnb_final_final['nombre_total_2023'] - bnb_final_final['nombre_total_2022']) / bnb_final_final['nombre_total_2022'],
    ecart_jours=bnb_final_final['jours_reserves_2023'] - bnb_final_final['jours_reserves_2022'],
    ecart_revenu=bnb_final_final['revenu_2023'] - bnb_final_final['revenu_2022'],
    ecart_revenu_jours=(bnb_final_final['revenu_2023'] / bnb_final_final['jours_reserves_2023']) - (bnb_final_final['revenu_2022'] / bnb_final_final['jours_reserves_2022'])
)

In [44]:
bnb_final_final_final

,code,echelle,nombre_reserv_120_2022,nombre_dispo_120_2022,nombre_total_2022,jours_reserves_2022,revenu_2022,jours_reserves_120_2022,revenu_120_2022,nombre_reserv_120_2023,...,revenu_2023,jours_reserves_120_2023,revenu_120_2023,evol_bnb_120,ecart_bnb_120,ecart_bnb_tot,evol_bnb_tot,ecart_jours,ecart_revenu,ecart_revenu_jours
0,34,departement,6233.0,15643.0,36444.0,2409136.0,290781081.0,1832776.0,215232204.0,5689.0,...,315493647.0,1829106.0,233359474.0,0.075881,1187.0,5407.0,0.148365,9670.0,24712566.0,9.734308
1,34001,commune,2.0,4.0,16.0,690.0,74625.0,455.0,48603.0,3.0,...,120060.0,793.0,81313.0,0.750000,3.0,3.0,0.187500,374.0,45435.0,4.686172
2,34002,commune,2.0,10.0,15.0,1028.0,177214.0,917.0,136338.0,3.0,...,228402.0,935.0,217818.0,0.100000,1.0,0.0,0.000000,-29.0,51188.0,56.243471
3,34003,commune,730.0,2663.0,5681.0,349231.0,39528819.0,264730.0,29208459.0,729.0,...,47218341.0,277452.0,33910387.0,0.079609,212.0,1228.0,0.216159,31245.0,7689522.0,10.915163
4,34004,commune,2.0,7.0,24.0,1457.0,885346.0,873.0,548588.0,10.0,...,1617070.0,2034.0,1459826.0,1.571429,11.0,4.0,0.166667,961.0,731724.0,61.113475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,343420000,iris,0.0,3.0,9.0,273.0,41789.0,172.0,23997.0,0.0,...,110938.0,340.0,89428.0,0.666667,2.0,1.0,0.111111,146.0,69149.0,111.695236
933,343430000,iris,0.0,1.0,20.0,683.0,105182.0,84.0,14349.0,0.0,...,124492.0,387.0,45836.0,5.000000,5.0,0.0,0.000000,99.0,19310.0,5.196931
934,343440101,iris,152.0,347.0,706.0,52884.0,4553978.0,41927.0,3639394.0,129.0,...,5062994.0,43516.0,3982483.0,0.167147,58.0,219.0,0.310198,1738.0,509016.0,6.578894
935,343440102,iris,280.0,650.0,1220.0,98197.0,8813928.0,78652.0,7049797.0,242.0,...,9882497.0,77856.0,7732345.0,0.055385,36.0,271.0,0.222131,471.0,1068569.0,10.401479


In [46]:
verif(bnb_final_final_final)

1 départements présents
Le dataframe ne contient pas de colonnes vides
17 EPCI présentes
Le dataframe ne contient pas de colonnes vides
341 communes présentes
Le dataframe ne contient pas de colonnes vides
578 IRIS présentes
Le dataframe ne contient pas de colonnes vides


## FILOSOFI

In [ ]:
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_com_2020.xlsx')
filosofi_com_2020 = pd.read_excel(chemin, sheet_name=0)
chemin = os.path.join(PATH_DATA_G, G_FILOSOFI, 'filosofi_epci_2020.xlsx')
filosofi_epci_2020 = pd.read_excel(chemin)


filosofi = pd.concat([filosofi_com_2020, filosofi_epci_2020], ignore_index=True)

verif(filosofi, 'CODGEO')

Hérault non présent
1239 EPCI présentes
Le dataframe contient 2 colonnes vides : ['nb_men', 'nb_pers']
319 communes présentes
Le dataframe ne contient pas de colonnes vides
IRIS non présentes


## INSEE Logement

In [53]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, 'base-ic-logement-2020 - Copie.xlsx')
ic_logement_2020 = pd.read_excel(chemin, sheet_name=0)
ic_logement_2020 = ic_logement_2020[['IRIS', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC', 'P20_RP_LOC', 'P20_RP_LOCHLMV']]

chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, 'base-ic-logement-2014 - Copie.xls')
ic_logement_2014 = pd.read_excel(chemin, sheet_name=0)
ic_logement_2014 = ic_logement_2014[['IRIS', 'P14_LOG', 'P14_RP', 'P14_RSECOCC', 'P14_LOGVAC', 'P14_RP_LOC', 'P14_RP_LOCHLMV']]

ic_insee = pd.merge(ic_logement_2020, ic_logement_2014, on=['IRIS'])
ic_insee.rename(columns={"IRIS": "code"}, inplace=True)


In [54]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "base-cc-logement - Copie.xlsx")

cc_logement_2020 = pd.read_excel(chemin, sheet_name=0)
cc_logement_2020 = cc_logement_2020[['CODGEO', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC', 'P20_RP_LOC', 'P20_RP_LOCHLMV']]

cc_logement_2014 = pd.read_excel(chemin, sheet_name=1)
cc_logement_2014 = cc_logement_2014[['CODGEO', 'P14_LOG', 'P14_RP', 'P14_RSECOCC', 'P14_LOGVAC', 'P14_RP_LOC', 'P14_RP_LOCHLMV']]

cc_insee = pd.merge(cc_logement_2020, cc_logement_2014, on="CODGEO")
cc_insee.rename(columns={'CODGEO': 'code'}, inplace=True)

In [55]:
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "epci_log_plus_2020.xlsx")
ep_logement_2020 = pd.read_excel(chemin)
chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "epci_log_plus_2014.xlsx")
ep_logement_2014 = pd.read_excel(chemin)

ep_logement_2020.columns = ["INSEE", "P20_LOG", "P20_RP", "P20_RSECOCC", "P20_LOGVAC", "P20_RP_LOC"]
ep_logement_2014.columns = ["INSEE", "nom", "P14_RP_LOC", "P14_RSECOCC", "P14_LOGVAC", "P14_LOG", "P14_RP"]

ep_logement_2020 = ep_logement_2020[["INSEE", "P20_LOG", "P20_RP", "P20_RSECOCC", "P20_LOGVAC", "P20_RP_LOC"]]
ep_logement_2014 = ep_logement_2014[["INSEE", "P14_LOG", "P14_RP", "P14_RSECOCC", "P14_LOGVAC", "P14_RP_LOC"]]

ep_insee = pd.merge(ep_logement_2020, ep_logement_2014, on="INSEE")
ep_insee.rename(columns={"INSEE": "code"}, inplace=True)

In [56]:
rp_logement = pd.concat([ic_insee, cc_insee, ep_insee], ignore_index=True)
verif(rp_logement)

Hérault non présent
164 EPCI présentes
Le dataframe contient 2 colonnes vides : ['P20_RP_LOCHLMV', 'P14_RP_LOCHLMV']
342 communes présentes
Le dataframe ne contient pas de colonnes vides
578 IRIS présentes
Le dataframe ne contient pas de colonnes vides


Calcul des variables supplémentaires
table_iris['point_RP'] = (table_iris['P20_RP'] / table_iris['P20_LOG']) - (table_iris['P14_RP'] / table_iris['P14_LOG'])
table_iris['deriv_RP'] = (table_iris['P20_RP'] - table_iris['P14_RP']) / table_iris['P14_RP']
table_iris['croissance_relative_RP_log'] = ((table_iris['P20_RP'] - table_iris['P14_RP']) / table_iris['P14_RP']) - ((table_iris['P20_LOG'] - table_iris['P14_LOG']) / table_iris['P14_LOG'])
table_iris['gain_plpr'] = (table_iris['P20_RP_LOC'] - table_iris['P20_RP_LOCHLMV']) - (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])
table_iris['taux_d_implantation'] = table_iris['nombre_dispo_120_2023'] / (table_iris['P20_LOG'] - table_iris['P20_RP'])
table_iris['taux_d_implantation_parc_tot'] = table_iris['nombre_total_2023'] / table_iris['P20_LOG']
table_iris['taux_d_implantation_parc_tot_120'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']
table_iris['potentiel_reconquete'] = table_iris['nombre_dispo_120_2023']
table_iris['taux_reconquete'] = table_iris['nombre_dispo_120_2023'] / (table_iris['P20_RP_LOC'] - table_iris['P14_RP_LOC'])
table_iris['deriv_plpr'] = ((table_iris['P20_RP_LOC'] - table_iris['P20_RP_LOCHLMV']) - (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])) / (table_iris['P14_RP_LOC'] - table_iris['P14_RP_LOCHLMV'])
table_iris['taux_bnb_parc_total'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']
table_iris['part_rs'] = table_iris['P20_RSECOCC'] / table_iris['P20_LOG']
table_iris['part_vac'] = table_iris['P20_LOGVAC'] / table_iris['P20_LOG']
table_iris['deriv_rs'] = (table_iris['P20_RSECOCC'] - table_iris['P14_RSECOCC']) / table_iris['P14_RSECOCC']
table_iris['deriv_vac'] = (table_iris['P20_LOGVAC'] - table_iris['P14_LOGVAC']) / table_iris['P14_LOGVAC']
table_iris['bnb_120_sur_log_tot'] = table_iris['nombre_dispo_120_2023'] / table_iris['P20_LOG']

## DV3F

### agrege (seulement pour com et epci)
https://cerema.app.box.com/v/dv3f-indicateurs?page=7

In [57]:
def read_and_filter(nom_fichier, annee, type='maison', echelle=False):
    """
    Variables:
        nom_fichier: NOM du fichier
        type: 'appart' ou 'maison'
        dep (valeur par défault False):
            True pour département (Hérault)
            False pour autre territoires de l'Hérault
    Entrée: CHEMIN vers un fichier
    Sortie:
        if dep=False:
            pd.DataFrame avec ['codgeo', f'prix_median_{type}_{annee}'] comme colonnes
        else:
            int, prix médian au département de l'hérault
    """
    # TYPE
    if type == 'maison':
        sheet_index = 2
        col_select = 'valeurfonc_median_cod111'
    elif type == 'appart':
        sheet_index = 5
        col_select = 'valeurfonc_median_cod121'
    # LECTURE
    if echelle=='com':
        chemin_debut = PATH_DATA_C
    else:
        chemin_debut = PATH_DATA_G
    chemin_entier = os.path.join(chemin_debut, PATH_DATA_DV3F, nom_fichier)
    df = pd.read_excel(chemin_entier, sheet_name=sheet_index)
    # SELECTION
    if echelle=='dep':
        df = df[df['dep'] == '34'][col_select].values[0]
    elif echelle=='epci':
        epci_34 = ['200071058', '243400819', '200066348', '243400736', '200042646',
                   '243400769', '243400694', '243400355', '200022986', '200042653',
                   '243400017', '200066355', '243400520', '243400470', '243400488',
                   '200017341', '200066553']
        epci_34_df = pd.DataFrame(epci_34, columns=['code_epci'])
        df = pd.merge(df, epci_34_df, left_on="epci", right_on="code_epci")
        df = df[["epci", col_select]]
        df.columns = ['codgeo', f'prix_median_{type}_{annee}']
    else:
        df = df[df['codgeo'].str.startswith('34')]
        df = df[['codgeo', col_select]].dropna()
        df.columns = ['codgeo', f'prix_median_{type}_{annee}']
    return df

#### epci

In [67]:
# Partie 4: DV3F ----
# https://cerema.app.box.com/v/dv3f-indicateurs?page=7
# Chemin du dossier de téléchargement, G:/ est un lecteur réseau ne recevant pas les fichiers lourds...
prix_median_maison_34_2020_2022 = read_and_filter("dv3f_indicateurs_departements_2020_2022.xlsx", "2020_2022", "maison", 'dep')
prix_median_appart_34_2020_2022 = read_and_filter("dv3f_indicateurs_departements_2020_2022.xlsx", "2020_2022", "appart", 'dep')

prix_median_maison_34_2022 = read_and_filter("dv3f_indicateurs_departements_2020_2022.xlsx", "2022", "maison", 'dep')
prix_median_appart_34_2022 = read_and_filter("dv3f_indicateurs_departements_2020_2022.xlsx", "2022", "appart", 'dep')

dv3f_maison_2020_2022 = read_and_filter("dv3f_indicateurs_epci_2020_2022.xlsx", "2020_2022", "maison", 'epci')

dv3f_appart_2020_2022 = read_and_filter("dv3f_indicateurs_epci_2020_2022.xlsx", "2020_2022", "appart", 'epci')

dv3f_maison_2022 = read_and_filter("dv3f_indicateurs_epci_2020_2022.xlsx", "2022", "maison", 'epci')

dv3f_appart_2022 = read_and_filter("dv3f_indicateurs_epci_2020_2022.xlsx", "2022", "appart", 'epci')

dv3f_maison_2020 = read_and_filter("dv3f_indicateurs_epci_2020.xlsx", "2020", "maison", 'epci')

dv3f_appart_2020 = read_and_filter("dv3f_indicateurs_epci_2020.xlsx", "2020", "appart", 'epci')

dv3f_maison_2014 = read_and_filter("dv3f_indicateurs_epci_2014.xlsx", "2014", "maison", 'epci')

dv3f_appart_2014 = read_and_filter("dv3f_indicateurs_epci_2014.xlsx", "2014", "appart", 'epci')

dv3f_maison = pd.merge(dv3f_maison_2014, dv3f_maison_2020, on="codgeo")
dv3f_maison["croissance_prix_maison"] = (dv3f_maison["prix_median_maison_2020"] - dv3f_maison["prix_median_maison_2014"]) / dv3f_maison["prix_median_maison_2014"]

dv3f_appart = pd.merge(dv3f_appart_2014, dv3f_appart_2020, on="codgeo")
dv3f_appart["croissance_prix_appart"] = (dv3f_appart["prix_median_appart_2020"] - dv3f_appart["prix_median_appart_2014"]) / dv3f_appart["prix_median_appart_2014"]

dv3f_maison_2019_2021 = read_and_filter("dv3f_indicateurs_epci_2019_2021.xlsx", "2019_2021", "maison", 'epci')

dv3f_appart_2019_2021 = read_and_filter("dv3f_indicateurs_epci_2019_2021.xlsx", "2019_2021", "appart", 'epci')

dv3f_maison_2013_2015 = read_and_filter("dv3f_indicateurs_epci_2013_2015.xlsx", "2013_2015", "maison", 'epci')

dv3f_appart_2013_2015 = read_and_filter("dv3f_indicateurs_epci_2013_2015.xlsx", "2013_2015", "appart", 'epci')

dv3f_maison_pas = pd.merge(dv3f_maison_2013_2015, dv3f_maison_2019_2021, on="codgeo")
dv3f_maison_pas["croissance_prix_maison_pas"] = (dv3f_maison_pas["prix_median_maison_2019_2021"] - dv3f_maison_pas["prix_median_maison_2013_2015"]) / dv3f_maison_pas["prix_median_maison_2013_2015"]

dv3f_appart_pas = pd.merge(dv3f_appart_2013_2015, dv3f_appart_2019_2021, on="codgeo")
dv3f_appart_pas["croissance_prix_appart_pas"] = (dv3f_appart_pas["prix_median_appart_2019_2021"] - dv3f_appart_pas["prix_median_appart_2013_2015"]) / dv3f_appart_pas["prix_median_appart_2013_2015"]

dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_pas, on="codgeo", suffixes=("", "_pas"))
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2022, on="codgeo", suffixes=("", "_2022"))
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2020_2022, on="codgeo", suffixes=("", "_2020_2022"))
dv3f_maison["prix_median_maison_2022_norm"] = dv3f_maison["prix_median_maison_2022"] / prix_median_maison_34_2022
dv3f_maison["prix_median_maison_2020_2022_norm"] = dv3f_maison["prix_median_maison_2020_2022"] / prix_median_maison_34_2020_2022

dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_pas, on="codgeo", suffixes=("", "_pas"))
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2022, on="codgeo", suffixes=("", "_2022"))
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2020_2022, on="codgeo", suffixes=("", "_2020_2022"))
dv3f_appart["prix_median_appart_2022_norm"] = dv3f_appart["prix_median_appart_2022"] / prix_median_appart_34_2022
dv3f_appart["prix_median_appart_2020_2022_norm"] = dv3f_appart["prix_median_appart_2020_2022"] / prix_median_appart_34_2020_2022

dv3f_prix_median = pd.merge(dv3f_maison, dv3f_appart, on="codgeo")

# Calculate the prix_median_2020_2022 column
dv3f_prix_median['prix_median_2020_2022'] = np.where(
    np.isnan(dv3f_prix_median['prix_median_appart_2020_2022']),
    dv3f_prix_median['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022,
    (dv3f_prix_median['prix_median_maison_2020_2022'] + dv3f_prix_median['prix_median_appart_2020_2022']) /
    (prix_median_maison_34_2020_2022 + prix_median_appart_34_2020_2022)
)

# Calculate the croissance_prix_pas column
dv3f_prix_median['croissance_prix_pas'] = np.where(
    np.isnan(dv3f_prix_median['croissance_prix_appart_pas']),
    dv3f_prix_median['croissance_prix_maison_pas'],
    (dv3f_prix_median['croissance_prix_maison_pas'] + dv3f_prix_median['croissance_prix_appart_pas']) / 2
)

# Calculate the croissance_prix column
dv3f_prix_median['croissance_prix'] = np.where(
    np.isnan(dv3f_prix_median['croissance_prix_appart']),
    dv3f_prix_median['croissance_prix_maison'],
    (dv3f_prix_median['croissance_prix_maison'] + dv3f_prix_median['croissance_prix_appart']) / 2
)

print("nb lignes : ", len(dv3f_prix_median))
print("colonnes : ", dv3f_prix_median.columns)
dv3f_prix_median_epci = dv3f_prix_median.copy(deep=True)
verif(dv3f_prix_median_epci, 'codgeo')

nb lignes :  17
colonnes :  Index(['codgeo', 'prix_median_maison_2014', 'prix_median_maison_2020',
       'croissance_prix_maison', 'prix_median_maison_2013_2015',
       'prix_median_maison_2019_2021', 'croissance_prix_maison_pas',
       'prix_median_maison_2022', 'prix_median_maison_2020_2022',
       'prix_median_maison_2022_norm', 'prix_median_maison_2020_2022_norm',
       'prix_median_appart_2014', 'prix_median_appart_2020',
       'croissance_prix_appart', 'prix_median_appart_2013_2015',
       'prix_median_appart_2019_2021', 'croissance_prix_appart_pas',
       'prix_median_appart_2022', 'prix_median_appart_2020_2022',
       'prix_median_appart_2022_norm', 'prix_median_appart_2020_2022_norm',
       'prix_median_2020_2022', 'croissance_prix_pas', 'croissance_prix'],
      dtype='object')
Hérault non présent
17 EPCI présentes
Le dataframe ne contient pas de colonnes vides
Commune non présentes
IRIS non présentes


#### com

In [68]:
dv3f_maison_2014 = read_and_filter("dv3f_prix_volumes_communes_2014.xlsx", 2014, echelle='com')
dv3f_maison_2020 = read_and_filter("dv3f_indicateurs_communes_2020.xlsx", 2020, echelle='com')
dv3f_maison_2022 = read_and_filter("dv3f_indicateurs_communes_2022.xlsx", 2022, echelle='com')
dv3f_maison_2013_2015 = read_and_filter("dv3f_indicateurs_communes_2013_2015.xlsx", '2013_2015', echelle='com')
dv3f_maison_2019_2021 = read_and_filter("dv3f_indicateurs_communes_2019_2021.xlsx", '2019_2021', echelle='com')
dv3f_maison_2020_2022 = read_and_filter("dv3f_indicateurs_communes_2020_2022.xlsx", '2020_2022', echelle='com')

dv3f_appart_2014 = read_and_filter("dv3f_indicateurs_communes_2014.xlsx", 2014, type='appart', echelle='com')
dv3f_appart_2020 = read_and_filter("dv3f_indicateurs_communes_2020.xlsx", 2020, type='appart', echelle='com')
dv3f_appart_2022 = read_and_filter("dv3f_indicateurs_communes_2022.xlsx", 2022, type='appart', echelle='com')
dv3f_appart_2013_2015 = read_and_filter("dv3f_indicateurs_communes_2013_2015.xlsx", '2013_2015', type='appart', echelle='com')
dv3f_appart_2019_2021 = read_and_filter("dv3f_indicateurs_communes_2019_2021.xlsx", '2019_2021', type='appart', echelle='com')
dv3f_appart_2020_2022 = read_and_filter("dv3f_indicateurs_communes_2020_2022.xlsx", '2020_2022', type='appart', echelle='com')


prix_median_maison_34_2022 = read_and_filter("dv3f_indicateurs_departements_2022.xlsx", 2022, echelle='dep')
prix_median_maison_34_2020_2022 = read_and_filter("dv3f_indicateurs_departements_2022.xlsx", '2020_2022', echelle='dep')

prix_median_appart_34_2022 = read_and_filter("dv3f_indicateurs_departements_2022.xlsx", 2022, 'appart', echelle='dep')
prix_median_appart_34_2020_2022 = read_and_filter("dv3f_indicateurs_departements_2022.xlsx", '2020_2022', 'appart', echelle='dep')

# Fusion des données et calcul de la croissance des prix
dv3f_maison = pd.merge(dv3f_maison_2014, dv3f_maison_2020, on='codgeo')
dv3f_maison['croissance_prix_maison'] = (dv3f_maison['prix_median_maison_2020'] - dv3f_maison['prix_median_maison_2014']) / dv3f_maison['prix_median_maison_2014']

dv3f_appart = pd.merge(dv3f_appart_2014, dv3f_appart_2020, on='codgeo')
dv3f_appart['croissance_prix_appart'] = (dv3f_appart['prix_median_appart_2020'] - dv3f_appart['prix_median_appart_2014']) / dv3f_appart['prix_median_appart_2014']

dv3f_maison_pas = pd.merge(dv3f_maison_2013_2015, dv3f_maison_2019_2021, on='codgeo')
dv3f_maison_pas['croissance_prix_maison_pas'] = (dv3f_maison_pas['prix_median_maison_2019_2021'] - dv3f_maison_pas['prix_median_maison_2013_2015']) / dv3f_maison_pas['prix_median_maison_2013_2015']

dv3f_appart_pas = pd.merge(dv3f_appart_2013_2015, dv3f_appart_2019_2021, on='codgeo')
dv3f_appart_pas['croissance_prix_appart_pas'] = (dv3f_appart_pas['prix_median_appart_2019_2021'] - dv3f_appart_pas['prix_median_appart_2013_2015']) / dv3f_appart_pas['prix_median_appart_2013_2015']

# Fusion des données DV3F pour les maisons et les appartements avec les années 2022 et 2020-2022
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_pas, how='outer', suffixes=('', '_pas'), on='codgeo')
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2022, how='outer', suffixes=('', '_2022'), on='codgeo')
dv3f_maison = pd.merge(dv3f_maison, dv3f_maison_2020_2022, how='outer', suffixes=('', '_2020_2022'), on='codgeo')
dv3f_maison['prix_median_maison_2022_norm'] = dv3f_maison['prix_median_maison_2022'] / prix_median_maison_34_2022
dv3f_maison['prix_median_maison_2020_2022_norm'] = dv3f_maison['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022

dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_pas, how='outer', suffixes=('', '_pas'), on='codgeo')
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2022, how='outer', suffixes=('', '_2022'), on='codgeo')
dv3f_appart = pd.merge(dv3f_appart, dv3f_appart_2020_2022, how='outer', suffixes=('', '_2020_2022'), on='codgeo')
dv3f_appart['prix_median_appart_2022_norm'] = dv3f_appart['prix_median_appart_2022'] / prix_median_appart_34_2022
dv3f_appart['prix_median_appart_2020_2022_norm'] = dv3f_appart['prix_median_appart_2020_2022'] / prix_median_appart_34_2020_2022

# Fusion des données pour les maisons et les appartements
dv3f_prix_median = pd.merge(dv3f_maison, dv3f_appart, how='outer', on='codgeo')

# Calcul des colonnes prix_median_2020_2022, croissance_prix_pas et croissance_prix
dv3f_prix_median['prix_median_2020_2022'] = np.where(
pd.isna(dv3f_prix_median['prix_median_appart_2020_2022']),
dv3f_prix_median['prix_median_maison_2020_2022'] / prix_median_maison_34_2020_2022,
(dv3f_prix_median['prix_median_maison_2020_2022'] + dv3f_prix_median['prix_median_appart_2020_2022']) / (prix_median_maison_34_2020_2022 + prix_median_appart_34_2020_2022)
)


dv3f_prix_median['croissance_prix_pas'] = dv3f_prix_median.apply(lambda row: row['croissance_prix_maison_pas'] if pd.isna(row['croissance_prix_appart_pas']) else (row['croissance_prix_maison_pas'] + row['croissance_prix_appart_pas']) / 2, axis=1)

dv3f_prix_median['croissance_prix'] = dv3f_prix_median.apply(lambda row: row['croissance_prix_maison'] if pd.isna(row['croissance_prix_appart']) else (row['croissance_prix_maison'] + row['croissance_prix_appart']) / 2, axis=1)

print("nb lignes : ", len(dv3f_prix_median))
print("colonnes : ", dv3f_prix_median.columns)
dv3f_prix_median_com = dv3f_prix_median.copy(deep=True)
verif(dv3f_prix_median, 'codgeo')

nb lignes :  275
colonnes :  Index(['codgeo', 'prix_median_maison_2014', 'prix_median_maison_2020',
       'croissance_prix_maison', 'prix_median_maison_2013_2015',
       'prix_median_maison_2019_2021', 'croissance_prix_maison_pas',
       'prix_median_maison_2022', 'prix_median_maison_2020_2022',
       'prix_median_maison_2022_norm', 'prix_median_maison_2020_2022_norm',
       'prix_median_appart_2014', 'prix_median_appart_2020',
       'croissance_prix_appart', 'prix_median_appart_2013_2015',
       'prix_median_appart_2019_2021', 'croissance_prix_appart_pas',
       'prix_median_appart_2022', 'prix_median_appart_2020_2022',
       'prix_median_appart_2022_norm', 'prix_median_appart_2020_2022_norm',
       'prix_median_2020_2022', 'croissance_prix_pas', 'croissance_prix'],
      dtype='object')
Hérault non présent
EPCI non présentes
275 communes présentes
Le dataframe ne contient pas de colonnes vides
IRIS non présentes


In [79]:
dv3f_prix_median = pd.concat([dv3f_prix_median_epci, dv3f_prix_median_com], axis=0)
verif(dv3f_prix_median, 'codgeo')

Hérault non présent
17 EPCI présentes
Le dataframe ne contient pas de colonnes vides
275 communes présentes
Le dataframe ne contient pas de colonnes vides
IRIS non présentes


### détail

In [80]:
chemin = os.path.join(PATH_DATA_C, C_CORRES, "corres_parcelle_iris.csv")
corres_parcelle_iris = pd.read_csv(chemin, sep='\t', header=0, dtype=str)
corres_parcelle_iris['code_iris'] = corres_parcelle_iris['code_iris'].astype('Int64')

def traitement_dvf(df, year, code):  
    df['id_par'] = df['l_idpar'].apply(lambda x: x[0]) # Prends la première parcelle. A voir si on peut faire mieux.
    df = df.merge(corres_parcelle_iris, on="id_par", how='left')
    df = df.merge(corres_iris_com_epci, on='code_iris')
    # calcul pour chaque iris et chaque codtypbien, le nombre et la médiane du prix de vente
    iris_codtype_compte_med = df.groupby([code, 'codtypbien']).agg(
            compte=('valeurfonc', 'count'),
            médiane=('valeurfonc', 'median')
        ).reset_index()
    # calcul pour chaque codtypbien, la médiane du prix de vente
    total_mediane_codtype = df.groupby('codtypbien')['valeurfonc'].median().reset_index().rename(columns={'valeurfonc': 'med_typbien'})
    # calcul pour chaque iris, le nombre total de prix de vente
    total_compte = iris_codtype_compte_med.groupby(code)['compte'].sum().reset_index().rename(columns={'compte': 'total_compte'})
    # fusion des 3 dataframes
    iris_codtype_compte_med_medtypbien = iris_codtype_compte_med.merge(total_mediane_codtype, on="codtypbien", how='left')
    iris_codtype_compte_med_medtypbien_totcompte = iris_codtype_compte_med_medtypbien.merge(total_compte, on=code, how='left')
    data = iris_codtype_compte_med_medtypbien_totcompte
    # calcul de l'indicateur
    data['indicateur'] = (data['compte'] / data['total_compte']) * (data['médiane'] / data['med_typbien'])
    # calcul de l'indicateur par iris
    ind_marche_iris = data[[code, 'indicateur']].groupby(code).sum().reset_index()

    return ind_marche_iris.rename(columns={code:'code', 'indicateur': f'indicateur_marche_{year}'})

In [81]:
engine_ancien = create_engine(CONNEXION_ANCIEN)

codtypbien_list = ('1112', '1113', '12121', '12122', '12125', '12123', '12124', '12131', '12132', '12135', '12133', '12134')
requete_type = f"""SELECT idmutation, anneemut, libtypbien, codtypbien, valeurfonc, sbati, l_idpar
    FROM dvf.mutation 
    WHERE codtypbien IN {codtypbien_list}
    """

requetes = {
    "2022": text(f"""
    {requete_type} 
    AND anneemut IN (2020, 2021, 2022)
    """),
    "2020": text(f"""
    {requete_type} 
    AND anneemut IN (2019, 2020, 2021)
    """),
    "2014": text(f"""
    {requete_type} 
    AND anneemut IN (2013, 2014, 2015)
    """)
}

with engine_ancien.connect() as connection:
    result_2022 = connection.execute(requetes['2022'])
    dvf_2022 = pd.DataFrame(result_2022.fetchall(), columns=result_2022.keys())

    result_2020 = connection.execute(requetes['2020'])
    dvf_2020 = pd.DataFrame(result_2020.fetchall(), columns=result_2020.keys())

    result_2014 = connection.execute(requetes['2014'])
    dvf_2014 = pd.DataFrame(result_2014.fetchall(), columns=result_2014.keys())



In [84]:
indicateur_marche_iris_2022 = traitement_dvf(dvf_2022, 2022, 'code_iris')
indicateur_marche_iris_2020 = traitement_dvf(dvf_2020, 2020, 'code_iris')
indicateur_marche_iris_2014 = traitement_dvf(dvf_2014, 2014, 'code_iris')

indicateur_marche_com_2022 = traitement_dvf(dvf_2022, 2022, 'code_com')
indicateur_marche_com_2020 = traitement_dvf(dvf_2020, 2020, 'code_com')
indicateur_marche_com_2014 = traitement_dvf(dvf_2014, 2014, 'code_com')

indicateur_marche_epci_2022 = traitement_dvf(dvf_2022, 2022, 'code_epci')
indicateur_marche_epci_2020 = traitement_dvf(dvf_2020, 2020, 'code_epci')
indicateur_marche_epci_2014 = traitement_dvf(dvf_2014, 2014, 'code_epci')

indicateur_marche_2022 = pd.concat([indicateur_marche_iris_2022, indicateur_marche_com_2022, indicateur_marche_epci_2022], axis=0)
indicateur_marche_2020 = pd.concat([indicateur_marche_iris_2020, indicateur_marche_com_2020, indicateur_marche_epci_2020], axis=0)
indicateur_marche_2014 = pd.concat([indicateur_marche_iris_2014, indicateur_marche_com_2014, indicateur_marche_epci_2014], axis=0)

indicateur_marche = indicateur_marche_2022.merge(indicateur_marche_2020, on="code", how='outer', suffixes=("", "_2020"))
indicateur_marche = indicateur_marche.merge(indicateur_marche_2014, on="code", how='outer', suffixes=("", "_2014"))
indicateur_marche["evol_indicateur_marche"] = indicateur_marche["indicateur_marche_2020"] - indicateur_marche["indicateur_marche_2014"]
print("nb lignes : ", len(indicateur_marche))
print("colonnes : ", indicateur_marche.columns)
verif(indicateur_marche)
indicateur_marche.head(3)

nb lignes :  753
colonnes :  Index(['code', 'indicateur_marche_2022', 'indicateur_marche_2020',
       'indicateur_marche_2014', 'evol_indicateur_marche'],
      dtype='object')
Hérault non présent
17 EPCI présentes
Le dataframe ne contient pas de colonnes vides
269 communes présentes
Le dataframe ne contient pas de colonnes vides
467 IRIS présentes
Le dataframe ne contient pas de colonnes vides


,code,indicateur_marche_2022,indicateur_marche_2020,indicateur_marche_2014,evol_indicateur_marche
0,34001,0.787574,0.660959,0.76283,-0.101871
1,34002,0.71949,0.746362,0.511923,0.234439
2,34003,0.837231,0.834856,0.898106,-0.06325


## TOURISME

In [85]:
G_TOURISME = 'tourisme'

# COMMUNES
chemin = os.path.join(PATH_DATA_G, G_TOURISME, 'insee_tourisme_com_2020.xlsx')
tourisme_com_2020 = pd.read_excel(chemin)
print("colonnes tour : ", tourisme_com_2020.columns)

chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "base-cc-logement - Copie.xlsx")
cc_logement_2020 = pd.read_excel(chemin, sheet_name=0)
cc_logement_2020 = cc_logement_2020[['CODGEO', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC', 'P20_RP_LOC', 'P20_RP_LOCHLMV']]
print("colonnes log : ", cc_logement_2020.columns)

tour_log_com = pd.merge(tourisme_com_2020, cc_logement_2020, on='CODGEO')
tour_log_com['tour_log'] = tour_log_com['camhot'] / tour_log_com['P20_LOG']
tour_log_com = tour_log_com[['CODGEO', 'tour_log']]
tour_log_com.columns = ['INSEE', 'tour_log']
# Nombre de batîment touristique / logement ----
tour_log_com.head(3)

# EPCI
chemin = os.path.join(PATH_DATA_G, G_TOURISME, 'insee_tourisme_epci_2020.xlsx')
tourisme_epci_2020 = pd.read_excel(chemin)
print("colonnes tour : ", tourisme_epci_2020.columns)

chemin = os.path.join(PATH_DATA_G, G_LOGEMENT, "epci_log_plus_2020.xlsx")
ep_logement_2020 = pd.read_excel(chemin)
ep_logement_2020.columns = ["CODGEO", "P20_LOG", "P20_RP", "P20_RSECOCC", "P20_LOGVAC", "P20_RP_LOC"]
print("colonnes log : ", ep_logement_2020.columns)

tour_log_epci = pd.merge(tourisme_epci_2020, ep_logement_2020, on='CODGEO')
tour_log_epci['tour_log'] = tour_log_epci['camhot'] / tour_log_epci['P20_LOG']
tour_log_epci = tour_log_epci[['CODGEO', 'tour_log']]
tour_log_epci.columns = ['INSEE', 'tour_log']
# Nombre de batîment touristique / logement ----
tour_log_epci.head(3)

# EPCI + COMMUNES
tour_log = pd.concat([tour_log_com, tour_log_epci], axis=0)
tour_log.head(3)
print("nb lignes : ", len(tour_log))
print("colonnes : ", tour_log.columns)
verif(tour_log, 'INSEE')

colonnes tour :  Index(['CODEGEO', 'Nombre d'emplacements des campings',
       'Nombre d'emplacements loués à l’année', 'Nombre d'hôtels',
       'Nombre de campings', 'Nombre de chambres des hôtels',
       'Nombre d’emplacements offerts à la clientèle de passage', 'CODGEO',
       'camhot'],
      dtype='object')
colonnes log :  Index(['CODGEO', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC',
       'P20_RP_LOC', 'P20_RP_LOCHLMV'],
      dtype='object')
colonnes tour :  Index(['CODEGEO', 'Nombre d'emplacements des campings',
       'Nombre d'emplacements loués à l’année', 'Nombre d'hôtels',
       'Nombre de campings', 'Nombre de chambres des hôtels',
       'Nombre d’emplacements offerts à la clientèle de passage', 'CODGEO',
       'camhot'],
      dtype='object')
colonnes log :  Index(['CODGEO', 'P20_LOG', 'P20_RP', 'P20_RSECOCC', 'P20_LOGVAC',
       'P20_RP_LOC'],
      dtype='object')
nb lignes :  359
colonnes :  Index(['INSEE', 'tour_log'], dtype='object')
Hérault non présen

In [86]:
G_POPULATION = 'rp_pop'
chemin = os.path.join(PATH_DATA_G, G_POPULATION, 'trage_2020_geo2023_com_AVerser.xlsx')
pop_com_2020 = pd.read_excel(chemin)
print("colonnes pop : ", pop_com_2020.columns)

chemin = os.path.join(PATH_DATA_G, G_TOURISME, '2022_Carte_Offre_Marchande_Comm_Herault_Obs_Habitat.xlsx')
tourisme = pd.read_excel(chemin)
print("colonnes tourisme : ", tourisme.columns)
tourisme = tourisme[['CODE INSEE', 'NOMBRE TOTAL DE LITS']]
tourisme.columns = ['INSEE', 'nb_lit']
pop_tour = pd.merge(pop_com_2020, tourisme, left_on='CODEGEO', right_on='INSEE', how='left')
pop_tour['nb_lit'] = pop_tour['nb_lit'].fillna(0)
pop_tour['pop_tour'] = pop_tour['nb_lit'] / pop_tour['Ensemble']
pop_tour = pop_tour[['INSEE', 'pop_tour']]
# Nombre de lit / population ----
print("nb lignes : ", len(pop_tour))
print("colonnes : ", pop_tour.columns)
verif(pop_tour, 'INSEE')
pop_tour.head(3)

colonnes pop :  Index(['CODEGEO', 'Ensemble', '0_14', '15_29', '30_44', '45_59', '60_74',
       '75_89', '90_plus'],
      dtype='object')
colonnes tourisme :  Index(['CODE INSEE', 'COMMUNES HERAULTAISES', 'NOMBRE TOTAL de STRUCTURES',
       'NOMBRE TOTAL DE LITS'],
      dtype='object')
nb lignes :  342
colonnes :  Index(['INSEE', 'pop_tour'], dtype='object')
Hérault non présent
EPCI non présentes
342 communes présentes
Le dataframe ne contient pas de colonnes vides
IRIS non présentes


,INSEE,pop_tour
0,34001,0.000000
1,34002,0.000000
2,34003,0.982227
